# Waste Distribution

In [ ]:
import matplotlib.pyplot as plt
import geopandas as gpd
import pandas as pd
import re
import os

## Load data

In [ ]:
onedrive = os.path.join("/", "mnt", "c", "Users", "eschmann", "OneDrive - epfl.ch", "Research IT", "Advanced Services", "0042 – Blue City", "BlueCityViz")
SP04 = os.path.join(onedrive, "SP04_Waste")
waste_dir = os.path.join(SP04, "Waste collection points by types (density-based spatial clustering)")

sous_secteurs_filename = os.path.join(SP04, "Lausanne Districts.gpkg")
Household_waste_filename = os.path.join(waste_dir, "DI_final_clustered_centroids.csv")
paper_waste_filename = os.path.join(waste_dir, "PC_final_clustered_centroids.csv")
glass_waste_filename = os.path.join(waste_dir, "VE_final_clustered_centroids.csv")
Organic_waste_filename = os.path.join(waste_dir, "DV_final_clustered_centroids.csv")
stats_filename = os.path.join(SP04, "Waste generation per person in each neighborhood.xlsx")

sous_secteurs = gpd.read_file(sous_secteurs_filename)

Household_waste = gpd.read_file(Household_waste_filename)
paper_waste = gpd.read_file(paper_waste_filename)
glass_waste = gpd.read_file(glass_waste_filename)
Organic_waste = gpd.read_file(Organic_waste_filename)
stats = pd.read_excel(stats_filename, sheet_name="CSV")

In [ ]:
sous_secteurs["quartier_number"] = sous_secteurs["RefName"].str.extract(r'Quartier (\d+)')[0]
stats_mapping = stats.set_index(stats["Quartier"].str.extract(r'^(\d+)')[0])["Quartier"].to_dict()

sous_secteurs["RefName"] = sous_secteurs["quartier_number"].map(stats_mapping)

sous_secteurs.drop(columns=["quartier_number"], inplace=True)


stats.rename(columns={"household_total": "DI", "paper_total": "PC", "glass_total": "VE", "organic_total": "DV"}, inplace=True)

In [ ]:
def create_geometry(df):
    """
    Create a geometry column from the longitude and latitude columns of a DataFrame.
    """
    df["geometry"] = gpd.points_from_xy(df["centroid_lon"], df["centroid_lat"])

    df.drop(columns=["centroid_lon", "centroid_lat"], inplace=True)

    return df


## Create a map of the waste collection points in Lausanne, colored by type of waste.

In [ ]:
Household_waste = create_geometry(Household_waste)
paper_waste = create_geometry(paper_waste)
glass_waste = create_geometry(glass_waste)
Organic_waste = create_geometry(Organic_waste)

Household_waste["type"] = "DI"
paper_waste["type"] = "PC"
glass_waste["type"] = "VE"
Organic_waste["type"] = "DV"

total_waste = pd.concat([Household_waste, paper_waste, glass_waste, Organic_waste], ignore_index=True)

total_waste = gpd.GeoDataFrame(total_waste, geometry="geometry", crs="EPSG:4326")
total_waste.to_crs(sous_secteurs.crs, inplace=True)

## Join the waste collection points with the sous-secteurs.

In [ ]:
total_waste["quartier"] = total_waste.sjoin(sous_secteurs[["RefName", "geometry"]], how="left", predicate="within")["RefName"]
total_waste.dropna(subset="quartier", inplace=True)


In [ ]:
waste_types = total_waste["type"].unique()
Quartiers = total_waste["quartier"].unique()

for wt in waste_types:
    for q in Quartiers:
        count = total_waste[(total_waste["type"] == wt) & (total_waste["quartier"] == q)].shape[0]
        total_waste.loc[(total_waste["type"] == wt) & (total_waste["quartier"] == q), "amount"] = stats.loc[(stats["Quartier"] == q)][wt].values[0] / count

total_waste["amount_month"] = total_waste["amount"] / 12

In [ ]:
total_waste